In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]


{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/workspaces/llm-zoomcamp/02-vector-search/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [5]:
from google import genai
import os
from google.genai import types

gemini_client = genai.Client()
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash", contents="Is it too late to join the course?"
)
print(response.text)

Whether or not it's too late to join a course depends on several factors:

*   **When does the course start and end?** If the course has already ended, it is too late. If it's just started, you likely have time.
*   **Does the course have a registration deadline?** Some courses have deadlines after which you cannot enroll.
*   **Are there still spots available?** Even if the course hasn't started, it might be full.
*   **What is the course format?** For example, is the course instructor-led with fixed meeting times or is it self-paced?
*   **What material has already been covered?** If the course has progressed significantly, it might be difficult to catch up.
*   **Are there any prerequisites?** Missing the first weeks of a class may mean you did not learn the material needed to perform well in later classes.

To find out if it is too late, you should:

1.  **Check the course website or syllabus:** Look for start and end dates, registration deadlines, and any policies regarding late e

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
def llm(prompt):
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt
    )
    return response.text

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [13]:
rag('how do I run kafka?')


'To run kafka with Java, navigate to the project directory and run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nTo run kafka with Python, first create a virtual environment and run requirements.txt in that environment. To create a virtual env and install packages (run only once):\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you\'ll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it\'s env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.\nYou may need to run this command in terminal in the same directory (/docker/spark): chmod +x build.sh\nIf you encounter the error "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'", use p

## RAG with Vector Search


In [14]:
from qdrant_client import QdrantClient, models


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
qd_client = QdrantClient("http://localhost:6333")

In [16]:
EMBEDDING_DIMENSIONALITY = 512


In [17]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [18]:
collection_name = "zoomcamp-faq"


In [19]:
qd_client.delete_collection(collection_name=collection_name)


False

In [20]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [21]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [24]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [25]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
question = 'I just discovered the course. Can I still join it?'


In [26]:
question = 'I just discovered the course. Can I still join it?'


In [28]:
query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        limit=5,
        with_payload=True
    )

In [30]:
query_points

QueryResponse(points=[ScoredPoint(id=449, version=2, score=0.87408733, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=2, score=0.8603326, payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'questio

In [32]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [34]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [35]:
rag('how do I run kafka?')

vector_search is used


"There are different instructions depending on whether you are running kafka with Java or Python.\n\n**Java:**\n\nIn the project directory, run:\n`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`\n\nIf you have issues, make sure the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` is the correct server URL and that cluster key and secrets are updated in `src/main/java/org/example/Secrets.java`.\n\nIf you get a `kafka.errors.NoBrokersAvailable` error, your kafka broker docker container is likely not working. Use `docker ps` to confirm and then run `docker compose up -d` in the docker compose yaml file folder.\n\n**Python:**\n\nCreate a virtual environment and run `requirements.txt` in that environment.\n\nTo create a virtual env and install packages (run only once):\n`python -m venv env`\n`source env/bin/activate`\n`pip install -r ../requirements.txt`\n\nTo activate it (you'll need to run it every time you need the virtual env):\n`source env/bin/a